

    ██████╗ ██╗██╗     ███████╗████████╗███╗   ███╗     ██████╗██████╗  █████╗  ██████╗██╗  ██╗███████╗██████╗ 
    ██╔══██╗██║██║     ██╔════╝╚══██╔══╝████╗ ████║    ██╔════╝██╔══██╗██╔══██╗██╔════╝██║ ██╔╝██╔════╝██╔══██╗
    ██████╔╝██║██║     ███████╗   ██║   ██╔████╔██║    ██║     ██████╔╝███████║██║     █████╔╝ █████╗  ██████╔╝
    ██╔══██╗██║██║     ╚════██║   ██║   ██║╚██╔╝██║    ██║     ██╔══██╗██╔══██║██║     ██╔═██╗ ██╔══╝  ██╔══██╗
    ██████╔╝██║███████╗███████║   ██║   ██║ ╚═╝ ██║    ╚██████╗██║  ██║██║  ██║╚██████╗██║  ██╗███████╗██║  ██║
    ╚═════╝ ╚═╝╚══════╝╚══════╝   ╚═╝   ╚═╝     ╚═╝     ╚═════╝╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝╚═╝  ╚═╝╚══════╝╚═╝  ╚═╝
                                                                                                           



In [1]:
# import libraries
import boto3
import csv
import time
import random
import matplotlib.pyplot as plt
import os
import keras
import numpy as np
import tensorflow as tf

# sagemaker libraries
import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker.tuner      import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

Using TensorFlow backend.


In [2]:
# get the session information
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [8]:
# define S3 settings
bucket = 'blstm-cracker'
prefix = 'test-run'

# define the path to which the data will be uploaded
data_path     = 'lstm_cracker/data/dump.csv'
data_name     = 'train.csv'
key           = os.path.join(prefix, 'train', data_name)
s3_train_data = 's3://{}/{}'.format(bucket, key)

# define the output path
out         = os.path.join(prefix, 'output')
output_path = 's3://{}/{}'.format(bucket, out)

# upload the training data to S3
print('Uploading data to: {}'.format(s3_train_data))
boto3.resource('s3').Bucket(bucket).Object(key).put(Body=open(data_path, 'rb'))

# configure SageMaker input channel
input_data = {
    'training': sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', content_type='text/csv')
}

Uploading data to: s3://blstm-cracker/test-run/train/train.csv


In [4]:
!pygmentize lstm_cracker/program/program.py

# -*- coding: utf-8 -*-
"""

This module trains a bidirectional long short-term memory (LSTM) 
network on a dataset consisting solely of cleartext passwords.
The trained network is then used to predict the most likely
alterations and/or additions to a given sequence.

Example
-------
    To run the program, include the dataset containing the cleartext 
    passwords as the first argument. The code will handle the rest.

    $ python3 program.py

Notes
-----
    The dataset is assumed to contain two columns: the usernames and the 
    cleartext passwords.

    The network parameters (e.g., number of hidden units, embedding
    layer, etc.) are defined in the configuration file (config.yml).

    This is the basic flow of the code:

    1) read in data
        1) clean up data (duplicates, NaN, etc)
    2) get data characteristics
        1) determine number of characters
        2) determine/define longest sequence length
    3) generator
        3.1) tokenization
        3.2) sliding w

        tokenizer : 
            The Keras tokenizer object.
        ix_to_character : 
            The c-to-character dictionary.
        data : pd.DataFrame
            The dataset, including the tokenized passwords.

        Returns
        -------
        float
            The probability of the password.

        """

        # tokenize the password
        token  = self.tokenizer.texts_to_sequences([password])[0]
        x_test = DataGenerator.slide_window(token)
        x_test = np.array(x_test)
        y_test = token - 1

        # determine the probabilities of the permutations of the words
        probabilities = self.model.predict(x_test, verbose=0)

        # calculate the probability of the password
        password_probability = 0
        for index, probability in enumerate(probabilities):
            word                  = self.ix_to_character[y_test[index] + 1]  # the first element is <PAD>
            word_probability      = probability[y_test[index]]               # 

## generate estimator

In [9]:
tf_estimator = TensorFlow(entry_point='program.py', 
                          role=role,
                          source_dir='lstm_cracker/program',
                          model_dir=output_path,
                          train_instance_count=1, 
                          train_instance_type='local',
                          framework_version='1.12', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={'epochs': 1, 'training': s3_train_data}
                         )

In [10]:
tf_estimator.fit(input_data)

Creating tmpvs3dqwvv_algo-1-vws6y_1 ... 
Attaching to tmpvs3dqwvv_algo-1-vws6y_12mdone
algo-1-vws6y_1  | 2019-11-27 20:25:53,245 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-vws6y_1  | 2019-11-27 20:25:53,251 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-vws6y_1  | 2019-11-27 20:25:53,429 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-vws6y_1  | 2019-11-27 20:25:53,448 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-vws6y_1  | 2019-11-27 20:25:53,462 sagemaker-containers INFO     Invoking user script
algo-1-vws6y_1  | 
algo-1-vws6y_1  | Training Env:
algo-1-vws6y_1  | 
algo-1-vws6y_1  | {
algo-1-vws6y_1  |     "additional_framework_parameters": {},
algo-1-vws6y_1  |     "channel_input_dirs": {
algo-1-vws6y_1  |         "training": "/opt/ml/input/data/training"
algo-1-vws6y_1  |     },
algo-1-vws6y_1  |     "current_host": "alg

tmpvs3dqwvv_algo-1-vws6y_1 exited with code 1
Aborting on container exit...


RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpvs3dqwvv/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

In [ ]:
# include the date in the endpoint 
tf_endpoint_name = 'keras-tf-fmnist-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

# deploy the endpoint using GPU acceleration
tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                         instance_type='ml.c5.large',
                         accelerator_type='ml.eia1.medium',
                         endpoint_name=tf_endpoint_name)

## perform predictions

In [ ]:
 %matplotlib inline

num_samples = 5
indices = random.sample(range(x_val.shape[0] - 1), num_samples)
images = x_val[indices]/255
labels = y_val[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')
    
prediction = tf_predictor.predict(images.reshape(num_samples, 28, 28, 1))['predictions']
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)
print('Predicted labels are: {}'.format(predicted_label))

## Hyperparameter tuning

In [ ]:
hyperparameter_ranges = {
    'epochs':        IntegerParameter(20, 100),
    'learning-rate': ContinuousParameter(0.001, 0.1, scaling_type='Logarithmic'), 
    'batch-size':    IntegerParameter(32, 1024),
    'dense-layer':   IntegerParameter(128, 1024),
    'dropout':       ContinuousParameter(0.2, 0.6)
}

objective_metric_name = 'val_acc'
objective_type = 'Maximize'
metric_definitions = [{'Name': 'val_acc', 'Regex': 'val_acc: ([0-9\\.]+)'}]

tuner = HyperparameterTuner(tf_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=10,
                            max_parallel_jobs=2,
                            objective_type=objective_type)